In [1]:
import tensorflow as tf
import numpy as np
import importlib
from train import provider
from train import box_util

/home/vamsi/miniconda2/envs/py36/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
reload(box_util)
reload(provider)

<module 'train.provider' from 'train/provider.pyc'>

In [3]:
NUM_POINT=2048
TRAIN_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='train',\
                            rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [4]:
VAL_DATASET = provider.FrustumDataset(npoints=NUM_POINT, split='val',\
                            rotate_to_center=True, random_flip=False, random_shift=False, one_hot=True)

In [5]:
point_set,labels,c,d,e,f,g,h,i = TRAIN_DATASET.__getitem__(0)

In [6]:
labels.shape

(2048,)

In [7]:
def indices(window_len, stride_len, point_set, labels, corners_gt):
    IOU = []
    start = []
    end = []
    IOU_1 = []
    IOU_2 = []
    corner = []
    j = np.min(point_set,0)[2]
    i = np.min(point_set,0)[2]+window_len
    idtuple=set()
    while i<=np.max(point_set,0)[2]:
            max_ind = np.searchsorted(point_set[:,2], i)
            min_ind = np.searchsorted(point_set[:,2], j)
            if (min_ind,max_ind) not in idtuple and max_ind-min_ind>5:
                #print('fgbd')
                idtuple.add((min_ind,max_ind))
                labels_window = np.zeros_like(point_set[:,0])
                labels_window[min_ind:max_ind] = 1.0
                union = np.sum(np.logical_or(labels, labels_window))
                #print(union)
                inter = np.sum(np.logical_and(labels, labels_window))
                if (inter*1.0/union) < 0.3 or (inter*1.0/union) >=0.7 : # or 
                    window_points=point_set[min_ind:1+ max_ind][:, :3]
                    min_x, min_y, min_z = np.min(window_points,axis=0)
                    max_x, max_y, max_z = np.max(window_points,axis=0)
                    corners = [[max_x,max_y,max_z],[max_x,max_y,min_z],[min_x,max_y,min_z],[min_x,max_y,max_z],\
                               [max_x,min_y,max_z],[max_x,min_y,min_z],[min_x,min_y,min_z],[min_x,min_y,max_z]]
                    corners=np.array(corners)
                    corner.append(corners)
                    #print(corners)
                    iou_3d, IOU_2D,inter_area, inter_vol = box_util.box3d_iou(corners, corners_gt)
                    end.append(max_ind)
                    start.append(min_ind)
                    IOU.append(inter*1.0/union)
                    IOU_1.append(inter_area)
                    IOU_2.append(inter_vol)
            j = j+stride_len
            i = j+window_len
    #IOU = np.array(IOU)>0.7
    return np.array(start),np.array(end), np.array(IOU), np.array(IOU_1), np.array(IOU_2), np.array(corner)



In [47]:
def indices(window_len, stride_len, point_set, labels, corners_gt):
    IOU = []
    start = []
    end = []
    IOU_1 = []
    IOU_2 = []
    #corner = []
    j = np.min(point_set,0)[2]
    i = np.min(point_set,0)[2]+window_len
    idtuple=set()
    while i<=np.max(point_set,0)[2]:
            max_ind = np.searchsorted(point_set[:,2], i)
            min_ind = np.searchsorted(point_set[:,2], j)
            if (min_ind,max_ind) not in idtuple and max_ind-min_ind>5:
                #print('fgbd')
                idtuple.add((min_ind,max_ind))
                #labels_window = np.zeros_like(point_set[:,0])
                #labels_window[min_ind:max_ind] = 1.0
                #union = np.sum(np.logical_or(labels, labels_window))
                #print(union)
                #inter = np.sum(np.logical_and(labels, labels_window))
                #if (inter*1.0/union) < 0.3 or (inter*1.0/union) >=0.7 : # or 
                
                window_points=point_set[min_ind:1+ max_ind][:, :3]
                min_x, min_y, min_z = np.min(window_points,axis=0)
                max_x, max_y, max_z = np.max(window_points,axis=0)
                #corners = [[max_x,max_y,max_z],[max_x,max_y,min_z],[min_x,max_y,min_z],[min_x,max_y,max_z],\
                #           [max_x,min_y,max_z],[max_x,min_y,min_z],[min_x,min_y,min_z],[min_x,min_y,max_z]]
                corners = [[min_x,max_y,max_z],[min_x,max_y,min_z],[max_x,max_y,min_z],[max_x,max_y,max_z],\
                               [min_x,min_y,max_z],[min_x,min_y,min_z],[max_x,min_y,min_z],[max_x,min_y,max_z]]
                corners=np.array(corners)
                iou_3d, IOU_2D,inter_area, inter_vol = box_util.box3d_iou(corners, corners_gt)
                if inter_area<0.3 or inter_area>=0.6 :
                    #corner.append(corners)
                    #print(corners)
                    
                    end.append(max_ind)
                    start.append(min_ind)
                    #IOU.append(inter*1.0/union)
                    IOU_1.append(inter_area)
                    IOU_2.append(inter_vol)
            j = j+stride_len
            i = j+window_len
    IOU_1 = (np.array(IOU_1)>=0.6)
    return np.array(start),np.array(end), np.array(IOU), np.array(IOU_1), np.array(IOU_2)#, np.array(corner)


In [48]:
TRAIN_DATASET.get_center_view_box3d(1)

array([[ 0.63603565,  1.47      ,  8.95986384],
       [ 0.7288103 ,  1.47      ,  8.48891494],
       [-0.44856195,  1.47      ,  8.25697831],
       [-0.5413366 ,  1.47      ,  8.72792721],
       [ 0.63603565, -0.42      ,  8.95986384],
       [ 0.7288103 , -0.42      ,  8.48891494],
       [-0.44856195, -0.42      ,  8.25697831],
       [-0.5413366 , -0.42      ,  8.72792721]])

In [49]:
pedc = 0
key = 'train'
count = 0
count_0 = 0
count1 = 0
count2 = 0
count3 = 0
total = 0
m = 0
for m in range(TRAIN_DATASET.__len__()):
    
    point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(m)
    if cls[1]==1 :
        pedc += 1
        print(pedc)
        #print(point_set.shape)
        #print(np.min(point_set[:,2]))
        #print(point_set.shape)
        #point_set = TRAIN_DATASET.get_center_view_point_set(m)
        index = np.argsort(point_set[:,2])
        point_set = point_set[index]
        labels = labels[index]
        
        if key=='train':
            corners_gt = TRAIN_DATASET.get_center_view_box3d(m)
        if key=='val':
            corners_gt = VAL_DATASET.get_center_view_box3d(m)
            
        #print(point_set.shape)
         
        s,e,l,l1,l2 = indices(0.9,0.1,point_set,labels, corners_gt)
        
        #inm = np.argmax(l1)
        #print(inm)
        print(l1.shape)
        #print(l2[inm])
        #print(c[inm])

        if np.sum(l1>=0.7) >0:
            count += 1
        if np.sum(l1>=0.6) >0 :
            count_0 += 1
        if np.sum(l1>=0.5) >0:
            count1 += 1
            if np.sum(l2>=0.5) >0:
                count2 += 1
        if np.sum(l2>=0.5) >0:
            count3 += 1
        

1
(79,)
2
(87,)
3
(76,)
4
(79,)
5
(84,)
6
(21,)
7
(21,)
8
(23,)
9
(23,)
10
(23,)
11
(87,)
12
(83,)
13
(101,)
14
(128,)
15
(116,)
16
(109,)
17
(86,)
18
(78,)
19
(103,)
20
(83,)
21
(17,)
22
(11,)
23
(15,)
24
(16,)
25
(23,)
26
(24,)
27
(27,)
28
(25,)
29
(27,)
30
(25,)
31
(90,)
32
(70,)
33
(87,)
34
(66,)
35
(99,)
36
(53,)
37
(53,)
38
(45,)
39
(60,)
40
(55,)
41
(52,)
42
(69,)
43
(60,)
44
(64,)
45
(50,)
46
(82,)
47
(74,)
48
(105,)
49
(85,)
50
(101,)
51
(164,)
52
(121,)
53
(130,)
54
(118,)
55
(156,)
56
(16,)
57
(25,)
58
(23,)
59
(18,)
60
(24,)
61
(72,)
62
(72,)
63
(75,)
64
(72,)
65
(71,)
66
(95,)
67
(116,)
68
(108,)
69
(100,)
70
(98,)
71
(43,)
72
(42,)
73
(47,)
74
(46,)
75
(41,)
76
(31,)
77
(29,)
78
(29,)
79
(29,)
80
(29,)
81
(96,)
82
(102,)
83
(89,)
84
(108,)
85
(81,)
86
(11,)
87
(16,)
88
(17,)
89
(22,)
90
(13,)
91
(18,)
92
(21,)
93
(23,)
94
(22,)
95
(29,)
96
(116,)
97
(123,)
98
(141,)
99
(122,)
100
(141,)
101
(69,)
102
(87,)
103
(74,)
104
(60,)
105
(53,)
106
(110,)
107
(105,)
108
(105,)
109

KeyboardInterrupt: 

In [40]:
print(count, count_0, count1, count2, count3)

(0, 0, 0, 0, 0)


In [30]:
pedc = 0
key = 'train'
count = 0
count_0 = 0
count1 = 0
count2 = 0
count3 = 0
total = 0
m = 0
for m in range(TRAIN_DATASET.__len__()):
    
    point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(m)
    if cls[1]==1 :
        pedc += 1
        print(pedc)
        #print(point_set.shape)
        #print(np.min(point_set[:,2]))
        #print(point_set.shape)
        #point_set = TRAIN_DATASET.get_center_view_point_set(m)
        index = np.argsort(point_set[:,2])
        point_set = point_set[index]
        labels = labels[index]
        
        if key=='train':
            corners_gt = TRAIN_DATASET.get_center_view_box3d(m)
        if key=='val':
            corners_gt = VAL_DATASET.get_center_view_box3d(m)
            
        #print(point_set.shape)
         
        s,e,l,l1,l2, c = indices(1.0,0.1,point_set,labels, corners_gt)
        
        #inm = np.argmax(l1)
        #print(inm)
        #print(l1[inm])
        #print(l2[inm])
        #print(c[inm])

        if np.sum(l1>=0.7) >0:
            count += 1
        if np.sum(l1>=0.6) >0 :
            count_0 += 1
        if np.sum(l1>=0.5) >0:
            count1 += 1
            if np.sum(l2>=0.5) >0:
                count2 += 1
        if np.sum(l2>=0.5) >0:
            count3 += 1
        

(1562, 4)
1
(1559, 4)
2
(1507, 4)
3
(1515, 4)
4
(1606, 4)
5
(980, 4)
(1055, 4)
(1155, 4)
(1133, 4)
(1058, 4)
(184, 4)
(242, 4)
(211, 4)
(222, 4)
(186, 4)
(70, 4)
(78, 4)
(83, 4)
(82, 4)
(67, 4)
(242, 4)
(210, 4)
(241, 4)
(285, 4)
(235, 4)
(1201, 4)
(1512, 4)
(921, 4)
(1131, 4)
(1002, 4)
(1575, 4)
(1897, 4)
(1569, 4)
(1889, 4)
(1840, 4)
(104, 4)
6
(97, 4)
7
(98, 4)
8
(108, 4)
9
(118, 4)
10
(506, 4)
(507, 4)
(590, 4)
(492, 4)
(640, 4)
(213, 4)
(260, 4)
(266, 4)
(235, 4)
(218, 4)
(333, 4)
(288, 4)
(364, 4)
(296, 4)
(313, 4)
(151, 4)
(139, 4)
(158, 4)
(180, 4)
(156, 4)
(122, 4)
(114, 4)
(102, 4)
(101, 4)
(88, 4)
(40, 4)
(51, 4)
(46, 4)
(19, 4)
(45, 4)
(536, 4)
11
(471, 4)
12
(555, 4)
13
(601, 4)
14
(547, 4)
15
(582, 4)
16
(515, 4)
17
(469, 4)
18
(620, 4)
19
(531, 4)
20
(255, 4)
(224, 4)
(233, 4)
(268, 4)
(250, 4)
(66, 4)
21
(49, 4)
22
(59, 4)
23
(59, 4)
24
(66, 4)
25
(527, 4)
(524, 4)
(468, 4)
(501, 4)
(473, 4)
(136, 4)
26
(175, 4)
27
(131, 4)
28
(153, 4)
29
(143, 4)
30
(500, 4)
(426, 4)
(

(736, 4)
130
(732, 4)
131
(836, 4)
132
(690, 4)
133
(1805, 4)
(89, 4)
(88, 4)
(96, 4)
(88, 4)
(85, 4)
(592, 4)
(615, 4)
(557, 4)
(638, 4)
(702, 4)
(671, 4)
(885, 4)
(780, 4)
(649, 4)
(869, 4)
(430, 4)
(374, 4)
(424, 4)
(366, 4)
(519, 4)
(133, 4)
(167, 4)
(172, 4)
(152, 4)
(159, 4)
(181, 4)
(186, 4)
(169, 4)
(164, 4)
(162, 4)
(337, 4)
(306, 4)
(342, 4)
(110, 4)
(98, 4)
(89, 4)
(91, 4)
(106, 4)
(56, 4)
(66, 4)
(72, 4)
(63, 4)
(196, 4)
(192, 4)
(198, 4)
(183, 4)
(199, 4)
(304, 4)
(298, 4)
(255, 4)
(287, 4)
(255, 4)
(217, 4)
(242, 4)
(230, 4)
(230, 4)
(207, 4)
(1427, 4)
(1237, 4)
(1151, 4)
(1181, 4)
(1297, 4)
(579, 4)
(485, 4)
(570, 4)
(541, 4)
(470, 4)
(327, 4)
(318, 4)
(244, 4)
(256, 4)
(268, 4)
(146, 4)
(122, 4)
(135, 4)
(70, 4)
(72, 4)
(94, 4)
(74, 4)
(89, 4)
(36, 4)
(38, 4)
(44, 4)
(65, 4)
(68, 4)
(70, 4)
(957, 4)
(1310, 4)
(1215, 4)
(1256, 4)
(1077, 4)
(462, 4)
(431, 4)
(573, 4)
(497, 4)
(500, 4)
(1134, 4)
(1451, 4)
(1242, 4)
(1132, 4)
(1291, 4)
(1086, 4)
(1319, 4)
(1213, 4)
(1330, 4

(206, 4)
(204, 4)
(252, 4)
(220, 4)
(257, 4)
(25, 4)
(27, 4)
(32, 4)
(1562, 4)
(1672, 4)
(1987, 4)
(1519, 4)
(1290, 4)
(568, 4)
(607, 4)
(574, 4)
(563, 4)
(581, 4)
(582, 4)
(564, 4)
(496, 4)
(636, 4)
(649, 4)
(408, 4)
(373, 4)
(394, 4)
(354, 4)
(437, 4)
(331, 4)
(326, 4)
(283, 4)
(361, 4)
(264, 4)
(96, 4)
(81, 4)
(79, 4)
(73, 4)
(70, 4)
(286, 4)
(287, 4)
(338, 4)
(384, 4)
(364, 4)
(75, 4)
256
(73, 4)
257
(84, 4)
258
(77, 4)
259
(84, 4)
260
(30, 4)
(81, 4)
(96, 4)
(119, 4)
(103, 4)
(133, 4)
(80, 4)
(68, 4)
(92, 4)
(69, 4)
(83, 4)
(55, 4)
(73, 4)
(69, 4)
(68, 4)
(231, 4)
(208, 4)
(242, 4)
(228, 4)
(229, 4)
(1572, 4)
(1637, 4)
(1677, 4)
(1594, 4)
(1516, 4)
(1034, 4)
(1192, 4)
(1205, 4)
(970, 4)
(873, 4)
(236, 4)
(248, 4)
(227, 4)
(230, 4)
(211, 4)
(163, 4)
(101, 4)
(121, 4)
(138, 4)
(136, 4)
(66, 4)
(59, 4)
(66, 4)
(51, 4)
(82, 4)
(600, 4)
261
(606, 4)
262
(514, 4)
263
(603, 4)
264
(546, 4)
265
(284, 4)
266
(213, 4)
267
(286, 4)
268
(222, 4)
269
(269, 4)
270
(281, 4)
271
(282, 4)
272
(296

(194, 4)
372
(254, 4)
373
(191, 4)
374
(228, 4)
375
(390, 4)
376
(459, 4)
377
(414, 4)
378
(468, 4)
379
(442, 4)
380
(248, 4)
381
(289, 4)
382
(250, 4)
383
(271, 4)
384
(208, 4)
385
(795, 4)
386
(840, 4)
387
(801, 4)
388
(802, 4)
389
(895, 4)
390
(313, 4)
(381, 4)
(467, 4)
(376, 4)
(325, 4)
(36, 4)
391
(35, 4)
392
(29, 4)
393
(34, 4)
394
(29, 4)
395
(1245, 4)
396
(1428, 4)
397
(1354, 4)
398
(1417, 4)
399
(1378, 4)
400
(1000, 4)
(1226, 4)
(1006, 4)
(1126, 4)
(1092, 4)
(1105, 4)
(1159, 4)
(996, 4)
(1188, 4)
(982, 4)
(125, 4)
(162, 4)
(134, 4)
(149, 4)
(173, 4)
(1309, 4)
(1561, 4)
(1236, 4)
(1527, 4)
(1587, 4)
(547, 4)
401
(607, 4)
402
(653, 4)
403
(550, 4)
404
(607, 4)
405
(164, 4)
(140, 4)
(119, 4)
(699, 4)
(833, 4)
(846, 4)
(852, 4)
(746, 4)
(174, 4)
406
(174, 4)
407
(185, 4)
408
(165, 4)
409
(154, 4)
410
(101, 4)
411
(112, 4)
412
(120, 4)
413
(102, 4)
414
(101, 4)
415
(83, 4)
416
(86, 4)
417
(91, 4)
418
(67, 4)
419
(88, 4)
420
(38, 4)
421
(32, 4)
422
(38, 4)
423
(34, 4)
424
(38, 4)
42

(203, 4)
595
(173, 4)
596
(160, 4)
597
(148, 4)
598
(136, 4)
599
(144, 4)
600
(150, 4)
(170, 4)
(168, 4)
(168, 4)
(131, 4)
(43, 4)
601
(43, 4)
602
(49, 4)
603
(42, 4)
604
(54, 4)
605
(2028, 4)
(73, 4)
606
(67, 4)
607
(75, 4)
608
(74, 4)
609
(85, 4)
610
(463, 4)
(336, 4)
(397, 4)
(406, 4)
(469, 4)
(91, 4)
(94, 4)
(95, 4)
(102, 4)
(89, 4)
(279, 4)
(325, 4)
(304, 4)
(300, 4)
(282, 4)
(165, 4)
(195, 4)
(186, 4)
(154, 4)
(175, 4)
(209, 4)
(268, 4)
(220, 4)
(253, 4)
(201, 4)
(138, 4)
(189, 4)
(167, 4)
(142, 4)
(174, 4)
(161, 4)
(141, 4)
(180, 4)
(128, 4)
(170, 4)
(225, 4)
(177, 4)
(166, 4)
(148, 4)
(186, 4)
(1592, 4)
(1595, 4)
(1937, 4)
(1461, 4)
(1754, 4)
(775, 4)
(896, 4)
(969, 4)
(782, 4)
(817, 4)
(642, 4)
(677, 4)
(737, 4)
(696, 4)
(670, 4)
(361, 4)
(429, 4)
(349, 4)
(430, 4)
(427, 4)
(369, 4)
(375, 4)
(403, 4)
(379, 4)
(406, 4)
(354, 4)
611
(417, 4)
612
(383, 4)
613
(420, 4)
614
(487, 4)
615
(124, 4)
(99, 4)
(112, 4)
(80, 4)
(120, 4)
(147, 4)
(119, 4)
(103, 4)
(147, 4)
(138, 4)
(70, 4)


...

(1107, 4)
10117
(1430, 4)
(1141, 4)
(1514, 4)
(1343, 4)
(1570, 4)
(811, 4)
(856, 4)
(852, 4)
(894, 4)
(846, 4)
(658, 4)
(589, 4)
(585, 4)
(605, 4)
(734, 4)
(305, 4)
(314, 4)
(326, 4)
(312, 4)
(298, 4)
(113, 4)
(137, 4)
(130, 4)
(126, 4)
(343, 4)
(293, 4)
(250, 4)
(283, 4)
(251, 4)
(228, 4)
(203, 4)
(193, 4)
(211, 4)
(191, 4)
(33, 4)
10118
(40, 4)
10119
(45, 4)
10120
(26, 4)
10121
(41, 4)
10122
(124, 4)
(123, 4)
(136, 4)
(136, 4)
(137, 4)
(185, 4)
(223, 4)
(218, 4)
(179, 4)
(169, 4)
(206, 4)
(235, 4)
(264, 4)
(234, 4)
(212, 4)
(597, 4)
(558, 4)
(642, 4)
(548, 4)
(624, 4)
(160, 4)
(164, 4)
(165, 4)
(169, 4)
(155, 4)
(2021, 4)
(1910, 4)
(1695, 4)
(1847, 4)
(117, 4)
(108, 4)
(76, 4)
(109, 4)
(121, 4)
(23, 4)
(22, 4)
(18, 4)
(1169, 4)
(1138, 4)
(1043, 4)
(1205, 4)
(971, 4)
(786, 4)
(759, 4)
(900, 4)
(744, 4)
(735, 4)
(167, 4)
(213, 4)
(227, 4)
(202, 4)
(197, 4)
(489, 4)
(446, 4)
(459, 4)
(447, 4)
(456, 4)
(35, 4)
10123
(36, 4)
10124
(36, 4)
10125
(35, 4)
10126
(35, 4)
10127
(141, 4)
(133, 4

(79, 4)
10247
(102, 4)
(71, 4)
(84, 4)
(95, 4)
(107, 4)
(266, 4)
(364, 4)
(364, 4)
(316, 4)
(345, 4)
(102, 4)
(122, 4)
(114, 4)
(108, 4)
(139, 4)
(222, 4)
(270, 4)
(240, 4)
(254, 4)
(231, 4)
(141, 4)
(149, 4)
(97, 4)
(132, 4)
(108, 4)
(52, 4)
(63, 4)
(44, 4)
(472, 4)
(537, 4)
(349, 4)
(431, 4)
(536, 4)
(755, 4)
(763, 4)
(670, 4)
(595, 4)
(763, 4)
(306, 4)
(345, 4)
(224, 4)
(380, 4)
(264, 4)
(99, 4)
(104, 4)
(86, 4)
(88, 4)
(90, 4)
(98, 4)
(88, 4)
(93, 4)
(111, 4)
(104, 4)
(63, 4)
(48, 4)
(52, 4)
(235, 4)
(268, 4)
(255, 4)
(243, 4)
(229, 4)
(121, 4)
(109, 4)
(172, 4)
(159, 4)
(124, 4)
(162, 4)
(194, 4)
(146, 4)
(184, 4)
(194, 4)
(309, 4)
(306, 4)
(323, 4)
(362, 4)
(365, 4)
(390, 4)
(418, 4)
(405, 4)
(497, 4)
(457, 4)
(81, 4)
(93, 4)
(100, 4)
(124, 4)
(115, 4)
(1181, 4)
(1263, 4)
(1399, 4)
(1296, 4)
(1155, 4)
(1743, 4)
(1899, 4)
(1719, 4)
(314, 4)
(302, 4)
(246, 4)
(296, 4)
(289, 4)
(98, 4)
(111, 4)
(140, 4)
(145, 4)
(109, 4)
(62, 4)
(52, 4)
(35, 4)
(49, 4)
(47, 4)
(583, 4)
(536, 4)
(502

(576, 4)
10377
(696, 4)
(717, 4)
(585, 4)
(648, 4)
(657, 4)
(1225, 4)
(1252, 4)
(1051, 4)
(1100, 4)
(1035, 4)
(1172, 4)
(1210, 4)
(1225, 4)
(1440, 4)
(1243, 4)
(821, 4)
(1014, 4)
(841, 4)
(783, 4)
(858, 4)
(1042, 4)
(803, 4)
(1008, 4)
(1018, 4)
(922, 4)
(793, 4)
(765, 4)
(1084, 4)
(824, 4)
(821, 4)
(337, 4)
(369, 4)
(402, 4)
(331, 4)
(357, 4)
(275, 4)
(207, 4)
(217, 4)
(201, 4)
(232, 4)
(1322, 4)
(1191, 4)
(1317, 4)
(1383, 4)
(1163, 4)
(1062, 4)
(1195, 4)
(1546, 4)
(1267, 4)
(1532, 4)
(438, 4)
(434, 4)
(391, 4)
(412, 4)
(451, 4)
(265, 4)
(203, 4)
(261, 4)
(206, 4)
(234, 4)
(110, 4)
(124, 4)
(119, 4)
(119, 4)
(117, 4)
(2025, 4)
(1696, 4)
(1377, 4)
(1730, 4)
(1504, 4)
(1551, 4)
(269, 4)
(349, 4)
(294, 4)
(286, 4)
(311, 4)
(1436, 4)
(1064, 4)
(1106, 4)
(1060, 4)
(925, 4)
(907, 4)
(667, 4)
(667, 4)
(696, 4)
(795, 4)
(723, 4)
(603, 4)
(667, 4)
(843, 4)
(729, 4)
(81, 4)
(73, 4)
(81, 4)
(57, 4)
(69, 4)
(1057, 4)
10378
(968, 4)
10379
(1046, 4)
10380
(1112, 4)
10381
(1117, 4)
10382
(439, 4)
103

(476, 4)
10485
(518, 4)
10486
(511, 4)
10487
(101, 4)
10488
(71, 4)
10489
(93, 4)
10490
(93, 4)
10491
(88, 4)
10492
(143, 4)
10493
(137, 4)
10494
(122, 4)
10495
(127, 4)
10496
(124, 4)
10497
(61, 4)
(70, 4)
(66, 4)
(60, 4)
(69, 4)
(345, 4)
(365, 4)
(301, 4)
(328, 4)
(322, 4)
(335, 4)
(387, 4)
(431, 4)
(380, 4)
(288, 4)
(412, 4)
(469, 4)
(431, 4)
(433, 4)
(417, 4)
(1004, 4)
(1143, 4)
(1281, 4)
(1403, 4)
(1082, 4)
(1123, 4)
(1361, 4)
(1166, 4)
(1458, 4)
(1250, 4)
(85, 4)
(74, 4)
(83, 4)
(106, 4)
(73, 4)
(183, 4)
(235, 4)
(235, 4)
(241, 4)
(231, 4)
(927, 4)
(925, 4)
(1038, 4)
(838, 4)
(806, 4)
(655, 4)
(757, 4)
(636, 4)
(752, 4)
(624, 4)
(331, 4)
(305, 4)
(353, 4)
(332, 4)
(306, 4)
(225, 4)
(242, 4)
(187, 4)
(194, 4)
(193, 4)
(454, 4)
(508, 4)
(545, 4)
(408, 4)
(530, 4)
(85, 4)
(93, 4)
(104, 4)
(84, 4)
(90, 4)
(52, 4)
(72, 4)
(72, 4)
(63, 4)
(66, 4)
(284, 4)
(265, 4)
(283, 4)
(301, 4)
(268, 4)
(371, 4)
(392, 4)
(431, 4)
(370, 4)
(357, 4)
(312, 4)
(266, 4)
(286, 4)
(273, 4)
(285, 4)
(146, 

(811, 4)
(1246, 4)
(1548, 4)
(1027, 4)
(1122, 4)
(987, 4)
(1010, 4)
(959, 4)
(973, 4)
(889, 4)
(989, 4)
(1146, 4)
(846, 4)
(902, 4)
(667, 4)
(557, 4)
(646, 4)
(610, 4)
(488, 4)
(536, 4)
(376, 4)
(333, 4)
(334, 4)
(336, 4)
(348, 4)
(127, 4)
(155, 4)
(130, 4)
(127, 4)
(136, 4)
(101, 4)
(100, 4)
(101, 4)
(82, 4)
(98, 4)
(42, 4)
(39, 4)
(39, 4)
(49, 4)
(39, 4)
(297, 4)
(336, 4)
(319, 4)
(386, 4)
(365, 4)
(270, 4)
(273, 4)
(300, 4)
(282, 4)
(261, 4)
(1467, 4)
(1564, 4)
(1499, 4)
(1603, 4)
(1416, 4)
(1003, 4)
(975, 4)
(1006, 4)
(972, 4)
(1108, 4)
(266, 4)
(224, 4)
(211, 4)
(201, 4)
(233, 4)
(91, 4)
(88, 4)
(84, 4)
(96, 4)
(97, 4)
(529, 4)
(538, 4)
(490, 4)
(507, 4)
(611, 4)
(371, 4)
(358, 4)
(372, 4)
(381, 4)
(327, 4)
(409, 4)
(427, 4)
(461, 4)
(388, 4)
(340, 4)
(349, 4)
(332, 4)
(288, 4)
(327, 4)
(336, 4)
(1804, 4)
(1709, 4)
(2041, 4)
(1893, 4)
(2043, 4)
(972, 4)
(885, 4)
(950, 4)
(864, 4)
(993, 4)
(528, 4)
(686, 4)
(499, 4)
(530, 4)
(581, 4)
(278, 4)
(224, 4)
(235, 4)
(264, 4)
(255, 4)
(15

In [31]:
print(count, count_0, count1, count2, count3)

(6117, 8128, 9482, 8180, 8180)


In [50]:
# points in each window:2048
# number of windows
#

def get_list(start_idx,end_idx,num_points=1024,ensure_all=True):
    """
    param:
    start_idx  : list of indexes of the starting point in each window
    end_idx    : list of indees of ending point in each window
    NOTES: we used two lists because we ran into issues with tensorflow when we used tuples,
           this is a quick hack to avoid that
    num_points : number of points to be sampled from i.e number of points in each window
    ensure_all : in the case that there are fewwer than required points ensures all points are picked atleast once if selected as True
    
    
    returns:
    out:  an array of size 
          
          num_windowsxnum_pointsx1
          
          this will be used inside tensorflow to divide into windows 
          after we pass the points trough pointnet
    """
    
    out= []
    for i in range(np.shape(start_idx)[0]):
        
        if end_idx[i]-start_idx[i]+1<num_points:
            indexes=np.arange(start_idx[i],end_idx[i]+1)
            indexes2=np.random.choice(indexes, num_points-np.shape(indexes)[0],replace=True)
            indexes=np.concatenate((indexes,indexes2))
        else:
            indexes=np.random.choice(np.arange(start_idx[i],end_idx[i]+1),num_points,replace=False)
        out.append(np.reshape(indexes,(-1,1)))
    
    return np.array(out)


In [51]:
import tf_util #for conv2d
from model_util import NUM_HEADING_BIN, NUM_SIZE_CLUSTER, NUM_OBJECT_POINT
from model_util import point_cloud_masking, get_center_regression_net
from model_util import placeholder_inputs, parse_output_to_tensors, get_loss


In [52]:
# https://github.com/ailias/Focal-Loss-implement-on-Tensorflow/blob/master/focal_loss.py


from tensorflow.python.ops import array_ops

def focal_loss(sigmoid_p, target_tensor, weights=None, alpha=0.99, gamma=2):
    r"""Compute focal loss for predictions.
        Multi-labels Focal loss formula:
            FL = -alpha * (z-p)^gamma * log(p) -(1-alpha) * p^gamma * log
            (1-p)
                 ,which alpha = 0.25, gamma = 2, p = sigmoid(x), z = target_tensor.
    Args:
     prediction_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing the predicted logits for each class
     target_tensor: A float tensor of shape [batch_size, num_anchors,
        num_classes] representing one-hot encoded classification targets
     weights: A float tensor of shape [batch_size, num_anchors]
     alpha: A scalar tensor for focal loss alpha hyper-parameter
     gamma: A scalar tensor for focal loss gamma hyper-parameter
    Returns:
        loss: A (scalar) tensor representing the value of the loss function
    """
    #sigmoid_p = tf.nn.sigmoid(prediction_tensor)
    zeros = array_ops.zeros_like(target_tensor, dtype=target_tensor.dtype)
    
    # For poitive prediction, only need consider front part loss, back part is 0;
    # target_tensor > zeros <=> z=1, so poitive coefficient = z - p.
    pos_p_sub = array_ops.where(target_tensor > zeros, target_tensor - sigmoid_p, zeros)
    
    # For negative prediction, only need consider back part loss, front part is 0;
    # target_tensor > zeros <=> z=1, so negative coefficient = 0.
    neg_p_sub = array_ops.where(target_tensor > zeros, zeros, sigmoid_p)
    per_entry_cross_ent = - alpha * (pos_p_sub ** gamma) * tf.log(tf.clip_by_value(sigmoid_p, 1e-8, 1.0)) \
                          - (1.0 - alpha) * (neg_p_sub ** gamma) * tf.log(tf.clip_by_value(1.0 - sigmoid_p, 1e-8, 1.0))
    return tf.reduce_sum(per_entry_cross_ent)

In [53]:
# def classification_net(point_cloud, label_out_5, label_out_1, label_out_2, label_out_3, label_out_4, idx5, idx1, idx2, idx3, idx4, 
#                             is_training=True, bn_decay=True):
def classification_net(point_cloud, label_out_5,  idx5, 
                            is_training=True, bn_decay=True):
    '''
    3D PointNet network.
    param:
    
    point_cloud: TF tensor in shape (1,N,4)
                 frustum point clouds with XYZ and intensity in point channels
                 XYZs are in frustum coordinate
    label_out: TF tensor in shape (1,1)
            length-1 vector indicating the classification as car or background
    
    ??????????????????????????????????????????????????????????????????????????????????????
    
    is_training: TF boolean scalar
    bn_decay: TF float scalar
    end_points: dict                 SHOULD CHANGE THIS
    Output:                                         
    logits: TF tensor in shape (B,N,2), scores for bkg/clutter and object
    end_points: dict
    ??????????????????????????????????????????????????????????????????????????????????????
    '''
    
    batch_size = point_cloud.get_shape()[0].value #1 in our case
    num_point = point_cloud.get_shape()[1].value  #N in our case
  
    net = tf.expand_dims(point_cloud, 2)
    #print(net.get_shape())

    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv1', bn_decay=bn_decay)
    net = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv2', bn_decay=bn_decay)
    point_feat = tf_util.conv2d(net, 64, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv3', bn_decay=bn_decay)
    net = tf_util.conv2d(point_feat, 128, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv4', bn_decay=bn_decay)
    features = tf_util.conv2d(net, 1024, [1,1],
                         padding='VALID', stride=[1,1],
                         bn=True, is_training=is_training,
                         scope='conv5', bn_decay=bn_decay)
    
    features=tf.squeeze(features,axis=2)
    features=tf.squeeze(features,axis=0)   

    #features=tf.placeholder(tf.float32,[1,1024,4])

    def fn(x):
        return tf.gather_nd(features,x)
    
    windows_5 =tf.map_fn(fn,idx5,dtype=tf.float32)
#     windows_1 = tf.map_fn(fn,idx1,dtype=tf.float32)
#     windows_2 = tf.map_fn(fn,idx2,dtype=tf.float32)
#     windows_3 = tf.map_fn(fn,idx3,dtype=tf.float32)
#     windows_4 = tf.map_fn(fn,idx4,dtype=tf.float32)
    
    windows_5_1024 = tf.reduce_max(windows_5,axis=1)
#     windows_1_1024 = tf.reduce_max(windows_1,axis=1)
#     windows_2_1024 = tf.reduce_max(windows_2,axis=1)
#     windows_3_1024 = tf.reduce_max(windows_3,axis=1)
#     windows_4_1024 = tf.reduce_max(windows_4,axis=1)
    
    
    winex_5 =  tf.reshape(windows_5_1024,shape=[-1,1,1,1024])
#     winex_1 =  tf.reshape(windows_1_1024,shape=[-1,1,1,1024])
#     winex_2 =  tf.reshape(windows_2_1024,shape=[-1,1,1,1024])
#     winex_3 =  tf.reshape(windows_3_1024,shape=[-1,1,1,1024])
#     winex_4 =  tf.reshape(windows_4_1024,shape=[-1,1,1,1024])
   
    winex = tf.concat([winex_5], axis=0)
    label_out = tf.concat([label_out_5],axis=0)

    
    
#     winex = tf.concat([winex_5, winex_1, winex_2, winex_3, winex_4], axis=0)
#     label_out = tf.concat([label_out_5,label_out_1, label_out_2, label_out_3, label_out_4],axis=0)

    #classification head
    d1 = tf_util.conv2d(winex, 512, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense1', bn_decay=bn_decay)
    d2 = tf_util.conv2d(d1, 256, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='dense2', bn_decay=bn_decay)
    d2=tf.layers.dropout(d2,rate=0.5, training=is_training, name='drop1')
    
    logits = tf_util.conv2d(d2, 1, [1,1],
                             padding='VALID', stride=[1,1],
                             bn=True, is_training=is_training,
                             scope='out', bn_decay=bn_decay, activation_fn=None)
    
    logits = tf.squeeze(logits)
    logits=tf.reshape(tf.squeeze(logits),(1,-1))
    sig = tf.sigmoid(logits)

    #sig = tf.reshape(tf.squeeze(sig),(-1,1))
    y=tf.reshape(sig,[-1])
    
    max_ind = tf.argmax(y)
    
    tp = tf.equal(label_out[max_ind], 1)
    
    
    differentiable_round = tf.maximum(y-0.499,0)
    differentiable_round = differentiable_round * 10000
    y_pred = tf.minimum(differentiable_round, 1)
    #y_pred=tf.round(y)
    
    true_pos  = tf.reduce_sum(tf.cast(tf.logical_and(tf.cast(label_out,tf.bool),tf.cast(y_pred,tf.bool)),tf.float32))
    
    false_pos = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not(tf.cast(label_out,tf.bool)),tf.cast(y_pred,tf.bool)),tf.float32))
    true_neg  = tf.reduce_sum(tf.cast(tf.logical_not(tf.logical_or(tf.cast(label_out, tf.bool),\
                                                                   tf.cast(y_pred, tf.bool))),tf.float32))
    false_neg = tf.reduce_sum(tf.cast(tf.logical_and(tf.logical_not\
                                                     (tf.cast(y_pred,tf.bool)),tf.cast(label_out,tf.bool)),tf.float32))
    return sig,logits, label_out,true_pos,false_pos,true_neg,false_neg, tp, max_ind

In [57]:
tf.reset_default_graph()
with tf.device('/device:GPU:0'):
    num_points_5 =512
#     num_points_1 = 512
#     num_points_2 = 512
#     num_points_3 = 512
#     num_points_4 = 512
    
    inputs=tf.placeholder(tf.float32,[None,2048,4])#put placeholder

    idx_5 =tf.placeholder(tf.int32,[None,num_points_5,1])

#     idx_1 = tf.placeholder(tf.int32,[None,num_points_1,1])
    
#     idx_2 = tf.placeholder(tf.int32,[None,num_points_2,1])

#     idx_3 = tf.placeholder(tf.int32,[None,num_points_3,1])
    
#     idx_4 = tf.placeholder(tf.int32,[None,num_points_4,1])

    is_training = tf.placeholder(tf.bool)
    

    label_out_5 = tf.placeholder(tf.int32, [None])
#     label_out_1 = tf.placeholder(tf.int32, [None])
#     label_out_2 = tf.placeholder(tf.int32, [None])
#     label_out_3 = tf.placeholder(tf.int32, [None])
#     label_out_4 = tf.placeholder(tf.int32, [None])

    sig,logits, label_out,tp,fp,tn,fn, tp_num, maxi = classification_net(inputs, label_out_5, \
                                         idx_5, is_training)
    
    
#     sig,logits, label_out,tp,fp,tn,fn = classification_net(inputs, label_out_5, label_out_1,label_out_2, label_out_3, label_out_4, \
#                                          idx_5, idx_1, idx_2, idx_3, idx_4, is_training)
    
    func_fl = focal_loss(tf.squeeze(sig),tf.squeeze(tf.cast(label_out, tf.float32)), gamma=0.25, alpha=0.25)
    print(label_out.get_shape())
#     loss=tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
#     labels=tf.reshape(tf.cast(label_out,tf.float32),(-1,1)),
#     logits=tf.reshape(logits,(-1,1))))

    loss = tf.reduce_mean(func_fl)
#     tpr=tf.divide(tf.cast(tp,tf.float32),tf.add(tp,fp))
#     tnr=tf.divide(tf.cast(tn,tf.float32),tf.add(tn,fn))
#     loss=tf.reduce_sum(tf.constant(1.0)-tf.scalar_mul(0.5,tf.add(tpr,tnr)))
    
    
    optimizer = tf.train.AdamOptimizer(1e-5)
    #train_step=optimizer.minimize(loss)
    update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
    with tf.control_dependencies(update_ops):
        train_op = optimizer.minimize(loss)
    
    

(?,)


In [ ]:
from tensorflow.python import debug

In [ ]:
import datetime
import logging
from random import shuffle
#logging.basicConfig(filename='run_1.log',level=logging.DEBUG)
from tensorflow.python import debug as tf_debug
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))

IOU_list = []
saver = tf.train.Saver()
trix=[cc for cc in range(TRAIN_DATASET.__len__())]
shuffle(trix)
prob = []
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)) as sess:
    #sess=tf_debug.LocalCLIDebugWrapperSession(sess)
    sess.run(tf.global_variables_initializer())
    variables_names = [v.name for v in tf.trainable_variables()]
    values = sess.run(variables_names)
    for k, v in zip(variables_names, values):
        print "Variable: ", k
        print "Shape: ", v.shape
    # uncomment this when we want to load the checkpoint model
    #saver.restore(sess, "./checkpoints/model_new_labels_0_30005.ckpt")
    for j in range(0,15):   
        total_tp=0
        total_fp=0
        total_tn=0
        total_fn=0
        total_tpnums = 0
        total_fpnums = 0
        total_fnnums = 0
        
        
        print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S"))
        
        print('Epoch Number........................................',j)
        pedc=0
        
        for i in trix:
        #todo reformat code to have lists instead of many variable
            if TRAIN_DATASET.type_list[i]=='Pedestrian':
                pedc+=1
                if pedc%500==0:
                    total_tp=0
                    total_fp=0
                    total_tn=0
                    total_fn=0
                    total_tpnums = 0
                    total_fpnums = 0
                    total_fnnums = 0
                point_set,labels,a,b,c,d,e,f,cls = TRAIN_DATASET.__getitem__(i)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                corners_gt = TRAIN_DATASET.get_center_view_box3d(i)
                
                #print(point_set.shape)

                s5,e5,l,l1,l2 = indices(1.0,0.1,point_set,labels, corners_gt)
        
                #l1 = (l1>=0.6)
                
                print('Label shape', l1.shape)

#                 s1, e1, l1 = indices(i, 1.1, 0.2)

#                 s2, e2, l2 = indices(i, 1.3, 0.2)

#                 s3, e3, l3 = indices(i, 0.7, 0.1)

#                 s4, e4, l4 = indices(i, 0.5, 0.1)
                
                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)
#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                    point_set = np.expand_dims(point_set, axis=0)
                        
                    _, x,tpos,fpos,tneg,fneg, tpnums =sess.run([train_op, loss,tp,fp,tn,fn, tp_num],feed_dict={idx_5:ind_5,\
                                                               label_out_5: l1.astype(np.int32),\
                                                               inputs:point_set, is_training: True})
                    total_tp+=tpos
                    total_fp+=fpos
                    total_tn+=tneg
                    total_fn+=fneg
                    total_tpnums+= tpnums
                    total_fpnums+= (1-tpnums)
                    total_fnnums+= (1-tpnums)
                    print('----------------------------------------------------------------------------------')
                    print('Epoch ', j , 'Loss for the current example ',i,' = ', x,'pedc = ',pedc)
                    #logging.info('Loss for the current example ',i,' = ', x)
                    print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                    #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                    #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                    print('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                          ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    #logging.info('Cumulative metrics: Precision = ',(total_tp*1.0)/(total_tp+total_fp),\
                    #      ' Recall = ',(total_tp*1.0)/(total_tp+total_fn))
                    print('Single metrics: Precision = ',(total_tpnums*1.0)/(total_tpnums+total_fpnums),\
                          ' Recall = ',(total_tpnums*1.0)/(total_tpnums+total_fnnums))
                    print('----------------------------------------------------------------------------------')
                    
                    if(pedc%1000==0):
                        save_path = saver.save(sess, "./checkpoints/model_new_labels_"+str(j)+'_'+str(i)+".ckpt")
                    

                #print(c)
                #print(n)
            #i += 1
        #-----------------------------------------------------------------------------------------------#
        print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
        k = 0
        print(len(val[5]))
        Vtotal_tp=0
        Vtotal_fp=0
        Vtotal_tn=0
        Vtotal_fn=0
        Vtotal_tpnums = 0
        Vtotal_fpnums = 0
        Vtotal_fnnums = 0
        while k<len(val[5]):
        #todo reformat code to have lists instead of many variable

            if VAL_DATASET.type_list[k]=='Pedestrian':

                point_set,labels,a,b,c,d,e,f,cls = VAL_DATASET.__getitem__(k)

                index = np.argsort(point_set[:,2])
                point_set = point_set[index]
                labels = labels[index]

                
                corners_gt = VAL_DATASET.get_center_view_box3d(k)
                
                #print(point_set.shape)

                s5,e5,l,l1,l2 = indices(1.0,0.1,point_set,labels, corners_gt)
                
                #l1 = l1>0.5

#                 s1, e1, l1 = indices(k, 1.1, 0.2, 'val')

#                 s2, e2, l2 = indices(k, 1.3, 0.2, 'val')

#                 s3, e3, l3 = indices(k, 0.7, 0.1, 'val')

#                 s4, e4, l4 = indices(k, 0.5, 0.1, 'val')

                if s5.size :#and s2.size and s3.size and s4.size and s1.size:

                    ind_5 = get_list(s5,e5, num_points=512)

#                     ind_1 = get_list(s1,e1, num_points=512)

#                     ind_2 = get_list(s2,e2, num_points=512)

#                     ind_3 = get_list(s3,e3, num_points=512)

#                     ind_4 = get_list(s4,e4, num_points=512)

#                     t_windows=s1.shape[0]+s2.shape[0]+s3.shape[0]+s4.shape[0]+s5.shape[0]

                point_set = np.expand_dims(point_set, axis=0)
                print(Vtotal_tp)
                x,tpos,fpos,tneg,fneg, tp_nums, maxind =sess.run([loss,tp,fp,tn,fn, tp_num, maxi],feed_dict={idx_5:ind_5, \
                                                           label_out_5: l1.astype(np.int32), inputs:point_set, is_training: True})
                
            
                #window_points = ind_5[maxind]
            
                window_points = point_set[0,ind_5[maxind].reshape((-1))][:,:3]
                min_x, min_y, min_z = np.min(window_points,axis=0)
                max_x, max_y, max_z = np.max(window_points,axis=0)
                corners = [[min_x,max_y,max_z],[min_x,max_y,min_z],[max_x,max_y,min_z],[max_x,max_y,max_z],\
                           [min_x,min_y,max_z],[min_x,min_y,min_z],[max_x,min_y,min_z],[max_x,min_y,max_z]]
                corners=np.array(corners)
                corners_gt = VAL_DATASET.get_center_view_box3d(k)
                IOU, IOU_2D, inter, inter_vol = box_util.box3d_iou(corners, corners_gt)
                print('IOUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUUU')
                print(IOU, IOU_2D, inter, inter_vol)
                IOU_list.append([IOU, IOU_2D, inter, inter_vol, k])
                if inter>0.4:
                    c += 1
                    if inter>0.5:
                        c1 += 1
                if inter_vol>0.4:
                    c2 += 1
                    if inter_vol>0.5:
                        c3 +=1 
                
                print(Vtotal_tp)
                Vtotal_tp +=tpos
                Vtotal_fp +=fpos
                Vtotal_tn +=tneg
                Vtotal_fn +=fneg
                Vtotal_tpnums+= tp_nums
                Vtotal_fpnums+= (1-tp_nums)
                Vtotal_fnnums+= (1-tp_nums)
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')
                print('Epoch ', j , 'Loss for the current example ',k,' = ', x)
                #logging.info('Loss for the current example ',i,' = ', x)
                print('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)
                #logging.info('Metrics for current example: TP=',tpos,' FP=',fpos,' TN=',tneg,' FN=',fneg)  # will not print anything
                #print('Cumulative metrics: TP=',total_tp,' FP=',total_fp,' TN=',total_tn,' FN=',total_fn)
                print('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))

                print('Single metrics: Precision = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fpnums),\
                      ' Recall = ',(Vtotal_tpnums*1.0)/(Vtotal_tpnums+Vtotal_fnnums))
                #logging.info('Cumulative metrics: Precision = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fp),\
                #      ' Recall = ',(Vtotal_tp*1.0)/(Vtotal_tp+Vtotal_fn))
                print('&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&')

            k += 1

        #----------------------------------------------------------------------------------------------#
#         prob = sess.run([sig],feed_dict={idx_5:ind_5, idx_1:ind_1, idx_2:ind_2, idx_3:ind_3, idx_4:ind_4,\
#                                                        label_out_5: l5.astype(np.int32), label_out_1: l1.astype(np.int32),\
#                                                        label_out_2: l2.astype(np.int32), label_out_3: l3.astype(np.int32),\
#                                                        label_out_4: l4.astype(np.int32), inputs:point_set,  is_training: False})
#         print(prob)
        print(ind_5)
        print(label_out_5)
        prob = sess.run([sig],feed_dict={idx_5:ind_5,  \
                                                       label_out_5: l1.astype(np.int32), inputs:point_set,  is_training: True})
        print(prob)
print(datetime.datetime.now().strftime("%a, %d %B %Y %I:%M:%S") )

Tue, 15 May 2018 05:50:26
Variable:  conv1/weights:0
Shape:  (1, 1, 4, 64)
Variable:  conv1/biases:0
Shape:  (64,)
Variable:  conv1/bn/beta:0
Shape:  (64,)
Variable:  conv1/bn/gamma:0
Shape:  (64,)
Variable:  conv2/weights:0
Shape:  (1, 1, 64, 64)
Variable:  conv2/biases:0
Shape:  (64,)
Variable:  conv2/bn/beta:0
Shape:  (64,)
Variable:  conv2/bn/gamma:0
Shape:  (64,)
Variable:  conv3/weights:0
Shape:  (1, 1, 64, 64)
Variable:  conv3/biases:0
Shape:  (64,)
Variable:  conv3/bn/beta:0
Shape:  (64,)
Variable:  conv3/bn/gamma:0
Shape:  (64,)
Variable:  conv4/weights:0
Shape:  (1, 1, 64, 128)
Variable:  conv4/biases:0
Shape:  (128,)
Variable:  conv4/bn/beta:0
Shape:  (128,)
Variable:  conv4/bn/gamma:0
Shape:  (128,)
Variable:  conv5/weights:0
Shape:  (1, 1, 128, 1024)
Variable:  conv5/biases:0
Shape:  (1024,)
Variable:  conv5/bn/beta:0
Shape:  (1024,)
Variable:  conv5/bn/gamma:0
Shape:  (1024,)
Variable:  dense1/weights:0
Shape:  (1, 1, 1024, 512)
Variable:  dense1/biases:0
Shape:  (512,)
V

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 70467, ' = ', 14.774017, 'pedc = ', 15)
('Metrics for current example: TP=', 0.0, ' FP=', 17.0, ' TN=', 8.0, ' FN=', 2.0)
('Cumulative metrics: Precision = ', 0.04608294930875576, ' Recall = ', 0.5555555555555556)
('Single metrics: Precision = ', 0.06666666666666667, ' Recall = ', 0.06666666666666667)
----------------------------------------------------------------------------------
('Label shape', (98,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 14735, ' = ', 48.91601, 'pedc = ', 16)
('Metrics for current example: TP=', 4.0, ' FP=', 48.0, ' TN=', 44.0, ' FN=', 2.0)
('Cumulative metrics: Precision = ', 0.04938271604938271, ' Recall = ', 0.5714285714285714)
('Single metrics: Precision = ', 0.0625, ' Recall = ', 0.0625)
------------------------------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 2373, ' = ', 21.418306, 'pedc = ', 31)
('Metrics for current example: TP=', 2.0, ' FP=', 18.0, ' TN=', 27.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.06462212486308871, ' Recall = ', 0.6941176470588235)
('Single metrics: Precision = ', 0.0967741935483871, ' Recall = ', 0.0967741935483871)
----------------------------------------------------------------------------------
('Label shape', (56,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 11843, ' = ', 29.971226, 'pedc = ', 32)
('Metrics for current example: TP=', 0.0, ' FP=', 25.0, ' TN=', 31.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.0628997867803838, ' Recall = ', 0.6941176470588235)
('Single metrics: Precision = ', 0.09375, ' Recall = ', 0.09375)
------------------------------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 52670, ' = ', 25.302088, 'pedc = ', 47)
('Metrics for current example: TP=', 2.0, ' FP=', 29.0, ' TN=', 20.0, ' FN=', 1.0)
('Cumulative metrics: Precision = ', 0.08449612403100776, ' Recall = ', 0.7517241379310344)
('Single metrics: Precision = ', 0.2127659574468085, ' Recall = ', 0.2127659574468085)
----------------------------------------------------------------------------------
('Label shape', (21,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 64338, ' = ', 8.705663, 'pedc = ', 48)
('Metrics for current example: TP=', 2.0, ' FP=', 10.0, ' TN=', 9.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.08525345622119816, ' Recall = ', 0.7551020408163265)
('Single metrics: Precision = ', 0.22916666666666666, ' Recall = ', 0.22916666666666666)
------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 26391, ' = ', 27.657139, 'pedc = ', 63)
('Metrics for current example: TP=', 0.0, ' FP=', 30.0, ' TN=', 23.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.08815272318921955, ' Recall = ', 0.8010204081632653)
('Single metrics: Precision = ', 0.30158730158730157, ' Recall = ', 0.30158730158730157)
----------------------------------------------------------------------------------
('Label shape', (75,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 53757, ' = ', 39.994987, 'pedc = ', 64)
('Metrics for current example: TP=', 0.0, ' FP=', 40.0, ' TN=', 35.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.08621636463481604, ' Recall = ', 0.8010204081632653)
('Single metrics: Precision = ', 0.296875, ' Recall = ', 0.296875)
------------------------------------------------

...

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 27068, ' = ', 24.976057, 'pedc = ', 271)
('Metrics for current example: TP=', 0.0, ' FP=', 14.0, ' TN=', 32.0, ' FN=', 3.0)
('Cumulative metrics: Precision = ', 0.102669133192389, ' Recall = ', 0.9066511085180864)
('Single metrics: Precision = ', 0.5055350553505535, ' Recall = ', 0.5055350553505535)
----------------------------------------------------------------------------------
('Label shape', (55,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 4082, ' = ', 28.418907, 'pedc = ', 272)
('Metrics for current example: TP=', 1.0, ' FP=', 20.0, ' TN=', 32.0, ' FN=', 2.0)
('Cumulative metrics: Precision = ', 0.1025168006324944, ' Recall = ', 0.9046511627906977)
('Single metrics: Precision = ', 0.5036764705882353, ' Recall = ', 0.5036764705882353)
--------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 42370, ' = ', 27.738194, 'pedc = ', 287)
('Metrics for current example: TP=', 0.0, ' FP=', 25.0, ' TN=', 27.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10171428571428572, ' Recall = ', 0.9071347678369196)
('Single metrics: Precision = ', 0.5052264808362369, ' Recall = ', 0.5052264808362369)
----------------------------------------------------------------------------------
('Label shape', (29,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 42585, ' = ', 15.401469, 'pedc = ', 288)
('Metrics for current example: TP=', 0.0, ' FP=', 8.0, ' TN=', 21.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10161106177851072, ' Recall = ', 0.9071347678369196)
('Single metrics: Precision = ', 0.5034722222222222, ' Recall = ', 0.5034722222222222)
-----------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 49706, ' = ', 26.736803, 'pedc = ', 303)
('Metrics for current example: TP=', 0.0, ' FP=', 20.0, ' TN=', 30.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10266715381804896, ' Recall = ', 0.9113513513513514)
('Single metrics: Precision = ', 0.5115511551155115, ' Recall = ', 0.5115511551155115)
----------------------------------------------------------------------------------
('Label shape', (6,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 17280, ' = ', 3.1919782, 'pedc = ', 304)
('Metrics for current example: TP=', 0.0, ' FP=', 2.0, ' TN=', 4.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10264215268476805, ' Recall = ', 0.9113513513513514)
('Single metrics: Precision = ', 0.5098684210526315, ' Recall = ', 0.5098684210526315)
-------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 7627, ' = ', 47.548054, 'pedc = ', 319)
('Metrics for current example: TP=', 5.0, ' FP=', 34.0, ' TN=', 80.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10244241231624031, ' Recall = ', 0.9152016546018614)
('Single metrics: Precision = ', 0.5172413793103449, ' Recall = ', 0.5172413793103449)
----------------------------------------------------------------------------------
('Label shape', (80,))
----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 42933, ' = ', 34.803932, 'pedc = ', 320)
('Metrics for current example: TP=', 1.0, ' FP=', 30.0, ' TN=', 49.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10219146482122261, ' Recall = ', 0.9152892561983471)
('Single metrics: Precision = ', 0.51875, ' Recall = ', 0.51875)
---------------------------------------------------

----------------------------------------------------------------------------------
('Epoch ', 0, 'Loss for the current example ', 21132, ' = ', 16.01763, 'pedc = ', 335)
('Metrics for current example: TP=', 0.0, ' FP=', 12.0, ' TN=', 18.0, ' FN=', 0.0)
('Cumulative metrics: Precision = ', 0.10359361135758652, ' Recall = ', 0.9192913385826772)
('Single metrics: Precision = ', 0.5253731343283582, ' Recall = ', 0.5253731343283582)
----------------------------------------------------------------------------------
('Label shape', (99,))
